In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go 

In [19]:
#read the csv and store into pandas dataframe called graph
#change to data
data = pd.read_csv('RecBCD_plain.csv', header = None)

#get the index counter as a tangible column
ind=data.index

indx=ind[:, np.newaxis]

data['index'] =indx

#name the columns of the dataframe, one of the columns will be an index counter
data.columns = [ 'X position', 'Y position','index']

data=data[['index','X position','Y position']]

#find the mean of each column, store mean from each column in a variable

ave = data.mean(axis=0)
xave = ave.loc['X position']
yave = ave.loc['Y position']

# add a columns for displacement on x and y in pixel and nm coordinates 
# pixel displacement coordinates are found by subtracting the average of each column from each row respectively

#change movement to position
data["X displacement (pixels)"] = data['X position'] - xave
data["Y displacement (pixels)"] = data['Y position'] - yave
# Nanometer displacement found by multiplying 154 (1 pixel = 154nm)
#to each value in the pixel displacement columns and storing the action as a new column
data["X displacement (nm)"] = data['X displacement (pixels)']*154
data["Y displacement (nm)"]= data['Y displacement (pixels)']*154
#Time column is made by multiplying the index column by 20 as each time step is 20 ms
data["Time (ms)"] = data['index']*20

data.head()
#data = {'X displacment':[Xdis], 'Y displacement':[Ydis]}
#df = pd.DataFrame(data)


D:\anaconda3\lib\site-packages\pandas\core\indexes\range.py:720: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.



,index,X position,Y position,X displacement (pixels),Y displacement (pixels),X displacement (nm),Y displacement (nm),Time (ms)
0,0,989.52197,150.41046,0.298984,-0.089678,46.043521,-13.810418,0
1,1,989.46265,150.68140,0.239664,0.181262,36.908241,27.914342,20
2,2,989.48285,150.66527,0.259864,0.165132,40.019041,25.430322,40
3,3,989.51483,150.74721,0.291844,0.247072,44.943961,38.049082,60
4,4,989.52441,150.68672,0.301424,0.186582,46.419281,28.733622,80


In [4]:
# PK's down sampling'
#add every 3 points together, divide by 3, store this as one index

###  Simple moving average

In [20]:
# courtesy of https://www.datacamp.com/community/tutorials/moving-averages-in-pandas
# Take the nanometer displacement in X and Y as well as time and store into a separate data frame
ma = pd.DataFrame(data.iloc[:,0])
#Use the built in pandas function for simple moving average
#simple moving average takes the average of the selected value and values around it within a certain window
#For example the first value in pandas_SMA_3_time column is the result of taking the average of index 0 1 and 2 and 
#storing it in index 2 of the new column

# the window changes the number of values we average over
window = 3

ma['X movement' ] = data.iloc[:,1].rolling(window=window).mean()
ma['Y movement'] = data.iloc[:,2].rolling(window=window).mean()
ma['X displacement (pixels)'] = data.iloc[:,3].rolling(window=window).mean()
ma['Y displacement(pixels)'] = data.iloc[:,4].rolling(window=window).mean()
ma['X displacement (nm)'] = data.iloc[:,5].rolling(window=window).mean()
ma['Y displacement (nm)'] = data.iloc[:,6].rolling(window=window).mean()
ma['Time (ms)'] = data.iloc[:,7].rolling(window=window).mean()

ma = ma.apply (pd.to_numeric, errors='coerce')
ma = ma.dropna()
ma = ma.reset_index(drop=True)
ma

#put moving average into separate data frame


,index,X movement,Y movement,X displacement (pixels),Y displacement(pixels),X displacement (nm),Y displacement (nm),Time (ms)
0,2,989.489157,150.585710,0.266171,0.085572,40.990267,13.178082,20.0
1,3,989.486777,150.697960,0.263791,0.197822,40.623747,30.464582,40.0
2,4,989.507363,150.699733,0.284377,0.199595,43.794094,30.737676,60.0
3,5,989.531677,150.710373,0.308691,0.210235,47.538347,32.376236,80.0
4,6,989.558207,150.723087,0.335221,0.222949,51.623967,34.334089,100.0
...,...,...,...,...,...,...,...,...
1393,1395,988.927550,150.310180,-0.295436,-0.189958,-45.497159,-29.253538,27880.0
1394,1396,988.965067,150.287760,-0.257919,-0.212378,-39.719593,-32.706218,27900.0
1395,1397,988.985593,150.319853,-0.237393,-0.180285,-36.558486,-27.763844,27920.0
1396,1398,988.989397,150.290857,-0.233589,-0.209281,-35.972773,-32.229331,27940.0


### downsampling by averaging

In [23]:
#the goal is to find or create a fuction that takes the average of a certain number of indexes and stores
# this average as one value, then moves on to the next set of values
#for example the first new index should be the average of index 1, 2, and 3 the next new value
#should be the average of index 4, 5, and 6 and so on, furthermore, we want to be able to change the number of indexes 
#we average over.

#first, lets take a section of the original data frame to work with

da=pd.DataFrame(data.iloc[:,:])

da.head()

#copy the time column into a numpy array

#isolate the column (if we print this it will show as a dataframe with 2 cols: indexes and time values)
daT_column=da.iloc[:,7]
daDY_column=da.iloc[:,6]
daDX_column=da.iloc[:,5]
daPY_column=da.iloc[:,4]
daPX_column=da.iloc[:,3]
daI_column=da.iloc[:,0]
daX_column=da.iloc[:,1]
daY_column=da.iloc[:,2]
#We just want the values in the column
daT = daT_column.values
daDY = daDY_column.values
daDX = daDX_column.values
daPY = daPY_column.values
daPX = daPX_column.values
daI = daI_column.values
daX = daX_column.values
daY= daY_column.values
#This function taken from https://stackoverflow.com/questions/10847660/subsampling-averaging-over-a-numpy-array
# allows us to downsample by averages over a set number (change 'n' to the number of values you want to average over)
def average(arr, n):
    end =  n * int(len(arr)/n)
    return np.mean(arr[:end].reshape(-1, n), 1)
#Takes the time column from our 'da' dataframe and runs the function over it
#stores the new values in variables as an array (values in a row)

# can change the k parameter to chnage the bin size (I suggest to keep this as 3 for this kernal)
# If you want to change the bin size scroll down to other kernal
k = 3

#assigning each new row to a varialble
Time = average(daT,k)
Index = average(daI,k)
Xda = average(daX,k)
Yda = average(daY,k)
Ydisnm = average(daDY,k)
Xdisnm = average(daDX,k)
YdisP = average(daPY,k)
XdisP = average(daPX,k)


#reshapes the data in a 1D column
TimeT = Time[:, np.newaxis]
YdisnmT = Ydisnm[:, np.newaxis]
XdisnmT = Xdisnm[:, np.newaxis]
YdisPT = YdisP[:, np.newaxis]
XdisPT = XdisP[:,np.newaxis]
XdaT = Xda[:, np.newaxis]
YdaT = Yda[:,np.newaxis]
IndexT = Index[:,np.newaxis]

#stores in a new dataframe 'dsak' for: downsampling average k, k=bin size
dsak= pd.DataFrame(IndexT, columns=['Index'])
#appending to our data frame
dsak['X movement'] = XdaT
dsak['Y movement'] = YdaT
dsak['X displacement (pixels)'] = XdisPT
dsak['Y displacement (pixels)'] = YdisPT
dsak['X displacement (nm)'] = XdisnmT 
dsak['Y displacement (nm)'] = YdisnmT
dsak['Time (ms)'] = TimeT

dsak



,Index,X movement,Y movement,X displacement (pixels),Y displacement (pixels),X displacement (nm),Y displacement (nm),Time (ms)
0,1.0,989.489157,150.585710,0.266171,0.085572,40.990267,13.178082,20.0
1,4.0,989.531677,150.710373,0.308691,0.210235,47.538347,32.376236,80.0
2,7.0,989.520080,150.826983,0.297094,0.326845,45.752461,50.334176,140.0
3,10.0,989.528807,150.801360,0.305821,0.301222,47.096367,46.388182,200.0
4,13.0,989.495137,150.702787,0.272151,0.202649,41.911187,31.207889,260.0
...,...,...,...,...,...,...,...,...
461,1384.0,988.971700,150.310430,-0.251286,-0.189708,-38.698059,-29.215038,27680.0
462,1387.0,988.920657,150.325880,-0.302329,-0.174258,-46.558733,-26.835738,27740.0
463,1390.0,988.926390,150.316997,-0.296596,-0.183141,-45.675799,-28.203771,27800.0
464,1393.0,988.939207,150.317550,-0.283779,-0.182588,-43.702033,-28.118558,27860.0


### Plotting 

In [24]:
#downsampling by average (pixel displacement)
x2 = dsak.iloc[:,3]
y2 = dsak.iloc[:,4]
z2 = dsak.iloc[:,7] 

#original data (pixel displacement)
x3 = data.iloc[:,3]
y3 = data.iloc[:,4]
z3 = data.iloc[:,7] 

#simple moving average data
x4 = ma.iloc[:,3]
y4 = ma.iloc[:,4]
z4 = ma.iloc[:,7]




#must call go.Figure to have the most options for interactive plotting 

trace1=go.Scatter3d(x=x2, y=y2, z=z2, name = 'Downsampling by average', marker=dict(size=4,color='#1bcaf2',opacity=1, colorscale='Plasma', showscale=False, colorbar=dict(title='Time (ms)')),line=dict(color='#1bcaf2',width=2))
trace2=go.Scatter3d(x=x3, y=y3, z=z3, name = 'Original', marker=dict(size=4,color='white',opacity=0.8, showscale=False, colorbar=dict(title='Time (ms)')),line=dict(color='white',width=2))
trace3=go.Scatter3d(x=x4, y=y4, z=z4, name= 'Simple moving average', marker=dict(size=4,color='#f56e6e',opacity=0.7, showscale=False, colorbar=dict(title='Time (ms)')),line=dict(color='#f56e6e',width=2))


fig = go.Figure(data=[trace2,trace1,trace3])


fig.update_layout(title='RecBCD Original Data and Noise Corrections: Downsampling by an Average and SMA')

fig.update_layout(scene = dict(
                    xaxis_title='Movement on X (pixel displacement)',
                    yaxis_title='Movement on Y (pixel displacement)',
                    zaxis_title='Time (ms)'))

#It seems to be the norm to format long lines of code like this
#Here we can tweak the background color, grid color, and color of the origin for all axes/planes
fig.update_layout(scene = dict(
                    xaxis = dict(
                         backgroundcolor="black",
                         gridcolor="gray",
                         showbackground=True,
                         zerolinecolor="white",),
                    yaxis = dict(
                        backgroundcolor="black",
                        gridcolor="gray",
                        showbackground=True,
                        zerolinecolor="white"),
                    zaxis = dict(
                        backgroundcolor="black",
                        gridcolor="gray",
                        showbackground=True,
                        zerolinecolor="white"),),
                  )

#Here we can tweak the size and aspect ratio of the graph and determine the default camera zoom and angle 
fig.update_layout(
    width=800,
    height=700,
    autosize=False,
    scene=dict(
        camera=dict(
            up=dict(
                x=0,
                y=0,
                z=1
            ),
            eye=dict(
                x=1,
                y=2,
                z=2,
            )
        ),
        aspectratio = dict( x=1, y=1, z=4 ),
        aspectmode = 'manual'
    ),
)

fig.show()

### Change the bin size of the 'Downsampling by Average' filter

In [27]:
#Change the variable of our downsampling average
#change the value of c (must be an integer) to change the bin size of this filter 
c = 6

Timec = average(daT,c)
Indexc = average(daI,c)
Xdac = average(daX,c)
Ydac = average(daY,c)
Ydisnmc = average(daDY,c)
Xdisnmc = average(daDX,c)
YdisPc = average(daPY,c)
XdisPc = average(daPX,c)
#It is annoying that we have to change the 'n' to a different value everytime we want to change the sample average
# is it possible to assign to a variable?

#reshapes the data in a 1D column
TimeTc = Timec[:, np.newaxis]
YdisnmTc = Ydisnmc[:, np.newaxis]
XdisnmTc = Xdisnmc[:, np.newaxis]
YdisPTc = YdisPc[:, np.newaxis]
XdisPTc = XdisPc[:,np.newaxis]
XdaTc = Xdac[:, np.newaxis]
YdaTc = Ydac[:,np.newaxis]
IndexTc = Indexc[:,np.newaxis]

#stores in a new dataframe
dsac= pd.DataFrame(IndexTc, columns=['Index'])
#appending to our data frame
dsac['X movement'] = XdaTc
dsac['Y movement'] = YdaTc
dsac['X displacement (pixels)'] = XdisPTc
dsac['Y displacement (pixels)'] = YdisPTc
dsac['X displacement (nm)'] = XdisnmTc 
dsac['Y displacement (nm)'] = YdisnmTc
dsac['Time (ms)'] = TimeTc

dsac


,Index,X movement,Y movement,X displacement (pixels),Y displacement (pixels),X displacement (nm),Y displacement (nm),Time (ms)
0,2.5,989.510417,150.648042,0.287431,0.147904,44.264307,22.777159,50.0
1,8.5,989.524443,150.814172,0.301457,0.314034,46.424414,48.361179,170.0
2,14.5,989.349722,150.727872,0.126736,0.227734,19.517277,35.070979,290.0
3,20.5,989.209208,150.803983,-0.013778,0.303845,-2.121776,46.792176,410.0
4,26.5,989.234803,150.837933,0.011817,0.337795,1.819854,52.020476,530.0
...,...,...,...,...,...,...,...,...
228,1370.5,988.922810,150.320965,-0.300176,-0.179173,-46.227119,-27.592648,27410.0
229,1376.5,988.963602,150.321862,-0.259384,-0.178276,-39.945203,-27.454561,27530.0
230,1382.5,988.962453,150.336233,-0.260533,-0.163905,-40.122046,-25.241324,27650.0
231,1388.5,988.923523,150.321438,-0.299463,-0.178700,-46.117266,-27.519754,27770.0


In [28]:
# Change the variable of our simple moving average
maw = pd.DataFrame(data.iloc[:,0])
#Use the built in pandas function for simple moving average
#simple moving average takes the average of the selected value and values around it within a certain window
#For example the first value in pandas_SMA_3_time column is the result of taking the average of index 0 1 and 2 and 
#storing it in index 2 of the new column

# the window changes the number of values we average over
w = 10

maw['X movement' ] = data.iloc[:,1].rolling(window=w).mean()
maw['Y movement'] = data.iloc[:,2].rolling(window=w).mean()
maw['X displacement (pixels)'] = data.iloc[:,3].rolling(window=w).mean()
maw['Y displacement(pixels)'] = data.iloc[:,4].rolling(window=w).mean()
maw['X displacement (nm)'] = data.iloc[:,5].rolling(window=w).mean()
maw['Y displacement (nm)'] = data.iloc[:,6].rolling(window=w).mean()
maw['Time (ms)'] = data.iloc[:,7].rolling(window=w).mean()

maw = maw.apply (pd.to_numeric, errors='coerce')
maw = maw.dropna()
maw = maw.reset_index(drop=True)
maw

,index,X movement,Y movement,X displacement (pixels),Y displacement(pixels),X displacement (nm),Y displacement (nm),Time (ms)
0,9,989.503448,150.726317,0.280462,0.226179,43.191133,34.831560,90.0
1,10,989.508746,150.758458,0.285760,0.258320,44.007025,39.781274,110.0
2,11,989.522454,150.768142,0.299468,0.268004,46.118057,41.272610,130.0
3,12,989.524749,150.764127,0.301763,0.263989,46.471487,40.654300,150.0
4,13,989.517028,150.778028,0.294042,0.277890,45.282453,42.795054,170.0
...,...,...,...,...,...,...,...,...
1386,1395,988.928387,150.326687,-0.294599,-0.173451,-45.368261,-26.711460,27810.0
1387,1396,988.934612,150.319482,-0.288374,-0.180656,-44.409611,-27.821030,27830.0
1388,1397,988.948582,150.321147,-0.274404,-0.178991,-42.258231,-27.564620,27850.0
1389,1398,988.949009,150.316180,-0.273977,-0.183958,-42.192473,-28.329538,27870.0


In [29]:
#downsampling by average (pixel displacement)k=3
x2 = dsa3.iloc[:,3]
y2 = dsa3.iloc[:,4]
z2 = dsa3.iloc[:,7] 

#original data (pixel displacement) 
x3 = data.iloc[:,3]
y3 = data.iloc[:,4]
z3 = data.iloc[:,7] 

#simple moving average data window = 3
x4 = ma.iloc[:,3]
y4 = ma.iloc[:,4]
z4 = ma.iloc[:,7]

#downsampling by average c = 
x5 = dsac.iloc[:,3]
y5 = dsac.iloc[:,4]
z5 = dsac.iloc[:,7] 

#simple moving average data w = 
x6 = maw.iloc[:,3]
y6 = maw.iloc[:,4]
z6 = maw.iloc[:,7]

#must call go.Figure to have the most options for interactive plotting 

trace2=go.Scatter3d(x=x2, y=y2, z=z2, name = 'Downsampling by average (3)', marker=dict(size=4,color='lightskyblue', colorscale='Plasma', showscale=False, colorbar=dict(title='Time (ms)')),line=dict(color='lightskyblue',width=2))
trace1=go.Scatter3d(x=x3, y=y3, z=z3, name = 'Original', marker=dict(size=4,color='white',opacity=0.7, showscale=False, colorbar=dict(title='Time (ms)')),line=dict(color='white',width=2))
trace4=go.Scatter3d(x=x4, y=y4, z=z4, name= 'Simple moving average (3)', marker=dict(size=4,color='yellow',opacity=0.7, showscale=False, colorbar=dict(title='Time (ms)')),line=dict(color='yellow',width=2))
trace3=go.Scatter3d(x=x5, y=y5, z=z5, name= 'Downsampling by average c', marker=dict(size=4,color='mediumspringgreen',opacity=0.9, showscale=False, colorbar=dict(title='Time (ms)')),line=dict(color='mediumspringgreen',width=2))
trace5=go.Scatter3d(x=x6, y=y6, z=z6, name= 'Simple moving average w', marker=dict(size=4,color='#ff4f4f',opacity=0.7, showscale=False, colorbar=dict(title='Time (ms)')),line=dict(color='#ff4f4f',width=2))
fig = go.Figure(data=[trace1,trace2,trace3,trace4,trace5])


fig.update_layout(title='RecBCD Original Data and Noise Corrections: Downsampling by an Average and SMA')

fig.update_layout(scene = dict(
                    xaxis_title='Movement on X (pixel displacement)',
                    yaxis_title='Movement on Y (pixel displacement)',
                    zaxis_title='Time (ms)'))

#It seems to be the norm to format long lines of code like this
#Here we can tweak the background color, grid color, and color of the origin for all axes/planes
fig.update_layout(scene = dict(
                    xaxis = dict(
                         backgroundcolor="black",
                         gridcolor="gray",
                         showbackground=True,
                         zerolinecolor="white",),
                    yaxis = dict(
                        backgroundcolor="black",
                        gridcolor="gray",
                        showbackground=True,
                        zerolinecolor="white"),
                    zaxis = dict(
                        backgroundcolor="black",
                        gridcolor="gray",
                        showbackground=True,
                        zerolinecolor="white"),),
                  )

#Here we can tweak the size and aspect ratio of the graph and determine the default camera zoom and angle 
fig.update_layout(
    width=800,
    height=700,
    autosize=False,
    scene=dict(
        camera=dict(
            up=dict(
                x=0,
                y=0,
                z=1
            ),
            eye=dict(
                x=1,
                y=2,
                z=2,
            )
        ),
        aspectratio = dict( x=1, y=1, z=4 ),
        aspectmode = 'manual'
    ),
)

fig.show()

In [12]:
a = np.array([1,2,3,4,5,6,7,8,9,10])
d =  3 * int(len(a)/3)
b = a[:9].reshape(-1,3)

np.mean(b,1)


array([2., 5., 8.])

In [13]:
graph.head()

,index,X position,Y position,X displacement (pixels),Y displacement (pixels),X displacement (nm),Y displacement (nm),Time (ms)
0,0,989.52197,150.41046,0.298984,-0.089678,46.043521,-13.810418,0
1,1,989.46265,150.68140,0.239664,0.181262,36.908241,27.914342,20
2,2,989.48285,150.66527,0.259864,0.165132,40.019041,25.430322,40
3,3,989.51483,150.74721,0.291844,0.247072,44.943961,38.049082,60
4,4,989.52441,150.68672,0.301424,0.186582,46.419281,28.733622,80
